In [2]:
from tokenization_utils import *
import json
from tqdm.notebook import tqdm
import jsonlines

# bpe_model = FastBPEMode(codes=os.path.abspath(BPE_path), vocab_path=None)
# reloaded = torch.load(model_path, map_location="cpu")
# dico = Dictionary(reloaded["dico_id2word"], reloaded["dico_word2id"], reloaded["dico_counts"])
# dico = Dictionary.read_vocab(Roberta_BPE_path)



In [45]:
import os
import pickle
import re

fns = os.listdir(hyp_path)
for fn in fns:
    if fn.endswith("txt"):
        lang = lang_upper[fn.split('-')[1].split('_')[0]]
        detok = file_detokenizers[lang]
        with open(hyp_path + fn) as infile:
            lines = infile.readlines()
        detoked_list = []
        for line in lines:
            line = line.strip()
            detoked = detok(line)
            detoked = detoked.replace("else", "else ")
#             detoked = re.sub(r'\n(?=[^()]*})', '', detoked)
            if lang in ["C++", "C"]:
                detoked = detoked.replace(".h> ", ".h> \n").replace("> using", "> \nusing")
            if lang in ["PHP"]:
                detoked = detoked.replace("$ ", "$").replace("? >", "?>")
            detoked_list.append(detoked)
        detoked_path = hyp_path + "pickles/"
        if not os.path.exists(detoked_path):
            os.makedirs(detoked_path)
        with open(detoked_path + fn[:-3] + "pkl", 'wb') as outfile:
            pickle.dump(detoked_list, outfile)
    

In [15]:
import json

path = "../g4g/map_data/Java-program-tok.jsonl"

with open(path, 'r') as json_file:
    json_list = list(json_file)
snippets = []
for json_str in json_list:
    result = json.loads(json_str)
    snippets.append(result)

In [16]:
def func1(sid, tokens, bpe, j, errors, bpe_dict):
    for i in range(len(tokens)):
        tok = tokens[i]
        subtok = bpe[j]
        if tok == subtok:
            j += 1
            bpe_dict[tok] = [subtok]
        elif " " in tok:
            if tok == " " or tok == '" \\\n\t\t\t%(temp.data, succ.data)\n':
                continue
#             if "\\" in tok:
#                 print(tok)
#                 tok = tok.replace('\\\\', '\\').replace("\n", "").replace("\t", "")
#                 print(tok)
            tok_list = tok.strip().split(" ")
#             print(tok, tok_list, tokens)
            sub_bpe_dict = {}
            sub_errors = []
            j = func1(sid + " " + tok, tok_list, bpe, j, sub_errors, sub_bpe_dict)
            bpe_dict[tok] = []
            for tk in tok_list:
                bpe_dict[tok] += sub_bpe_dict[tk]
            if len(sub_errors) > 0:
                print("sub_errors", sub_errors)
        elif subtok.endswith("@@"):
            bpe_dict[tok] = []
            while subtok.endswith("@@"):
                bpe_dict[tok].append(subtok)
                j += 1
                subtok= bpe[j]
            bpe_dict[tok].append(subtok)
            j += 1
        else:
            errors.append(tuple([tok, subtok]))
            continue
        bpe_str = " ".join(bpe_dict[tok]).replace("@@ ", "")
        if bpe_str != tok:
            print("mismatch", sid, tok, "|", bpe_str)
    return j

In [21]:
def bpe_utility(sid, tokens, bpe, j, errors, bpe_dict):
    for i in range(len(tokens)):
        tok = tokens[i]
        subtok = bpe[j]
        if tok == subtok:
            j += 1
            bpe_dict[tok] = [subtok]
        elif " " in tok:
            if tok == " " or tok == '" \\\n\t\t\t%(temp.data, succ.data)\n':
                continue
            tok_list = tok.strip().split(" ")
            sub_bpe_dict = {}
            sub_errors = []
            j = bpe_utility(sid + " " + tok, tok_list, bpe, j, sub_errors, sub_bpe_dict)
            bpe_dict[tok] = []
            for tk in tok_list:
                bpe_dict[tok] += sub_bpe_dict[tk]
            if len(sub_errors) > 0:
                print("sub_errors", sub_errors)
        elif subtok.endswith("@@"):
            bpe_dict[tok] = []
            while subtok.endswith("@@"):
                bpe_dict[tok].append(subtok)
                j += 1
                subtok= bpe[j]
            bpe_dict[tok].append(subtok)
            j += 1
        else:
            errors.append(tuple([tok, subtok]))
            continue
        bpe_str = " ".join(bpe_dict[tok]).replace("@@ ", "")
        if bpe_str != tok:
            #print("mismatch", sid, tok, "|", bpe_str)
            pass
    return j
def createBPEDict(data_path):
    errors = []
    items = []
    with open(data_path, "r") as f:
        for line in f.readlines():
            items.append(json.loads(line))
    bpe_dict = {}
    for i, item in enumerate(items):
        j = 0
        bpe = item['bpe'].split(" ")
        tokens = item['tokens']
        bpe_utility(item['idx'], tokens, bpe, j, errors, bpe_dict)
    for key, value in bpe_dict.items():
        bpe_dict[key] = list(dict.fromkeys(value))
    return bpe_dict, errors

In [23]:
createBPEDict( "../g4g/map_data/Java-program-tok.jsonl")
a = 0

In [13]:
j = 0
errors = []
bpe_dict = {}
snippet = snippets[8531]
func1(snippet['idx'], snippet['tokens'], snippet['bpe'].split(), j, errors, bpe_dict)

IndexError: list index out of range

In [19]:
errors = []
bpe_dict = {}
for i, snippet in enumerate(snippets):
    print(i, snippet['idx'])
    j = 0
    func1(snippet['idx'], snippet['tokens'], snippet['bpe'].split(), j, errors, bpe_dict)

0 105-Java
1 1-Java
2 10-Java
3 100-Java
4 1008-Java
5 101-Java
6 1010-Java
7 1011-Java
8 1012-Java
9 1015-Java
10 102-Java
11 1021-Java
12 1022-Java
13 1023-Java
14 1025-Java
15 1028-Java
16 103-Java
17 1031-Java
18 1037-Java
19 1038-Java
20 1040-Java
21 1044-Java
22 1045-Java
23 1051-Java
24 1055-Java
25 1057-Java
26 1058-Java
27 1059-Java
28 106-Java
29 1060-Java
30 1061-Java
31 1064-Java
32 1065-Java
33 1066-Java
34 1067-Java
35 1068-Java
36 107-Java
37 1072-Java
38 1073-Java
39 1074-Java
40 1075-Java
41 1077-Java
42 1079-Java
43 108-Java
44 1080-Java
45 1081-Java
46 1082-Java
47 1083-Java
48 1085-Java
49 1087-Java
50 1089-Java
51 109-Java
52 1091-Java
53 1093-Java
54 1094-Java
55 1095-Java
56 1097-Java
57 1098-Java
58 110-Java
59 1109-Java
60 1110-Java
61 1111-Java
62 1112-Java
63 1113-Java
64 1114-Java
65 1115-Java
66 1117-Java
67 1118-Java
68 1119-Java
69 1124-Java
70 113-Java
71 1132-Java
72 1134-Java
73 1135-Java
74 1137-Java
75 114-Java
76 115-Java
77 1159-Java
78 116-Java
79

1087 26-Java
1088 260-Java
1089 2600-Java
1090 2601-Java
1091 2602-Java
1092 2604-Java
1093 2605-Java
1094 2606-Java
1095 2607-Java
1096 2608-Java
1097 2609-Java
1098 261-Java
1099 2611-Java
1100 2612-Java
1101 2613-Java
1102 2614-Java
1103 2615-Java
1104 2616-Java
1105 2617-Java
1106 2618-Java
1107 2619-Java
1108 262-Java
1109 2622-Java
1110 2623-Java
1111 2624-Java
1112 2625-Java
1113 2626-Java
1114 2628-Java
1115 2631-Java
1116 264-Java
1117 2641-Java
1118 2642-Java
1119 2643-Java
1120 2645-Java
1121 265-Java
1122 2652-Java
1123 2653-Java
1124 2655-Java
1125 2656-Java
1126 2657-Java
1127 2658-Java
1128 2659-Java
1129 266-Java
1130 2661-Java
1131 2662-Java
1132 2663-Java
1133 2664-Java
1134 2665-Java
1135 2667-Java
1136 2668-Java
1137 267-Java
1138 2670-Java
1139 2671-Java
1140 2672-Java
1141 2673-Java
1142 2674-Java
1143 2675-Java
1144 2676-Java
1145 2677-Java
1146 2679-Java
1147 2680-Java
1148 2681-Java
1149 2682-Java
1150 2683-Java
1151 2684-Java
1152 2686-Java
1153 2689-Java
1154